# Libraries

In [16]:
import pandas as pd 
import numpy as np 
import re 

In [18]:
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

# Constants

In [105]:
DATA_PATH = '../DATA/news_125k-tokens-relevant.csv'

# Bespoke Functions

# Import data

In [110]:
df = pd.read_csv(DATA_PATH, index_col=0)
df.head()

,title,body,topic,votes,clicks,comments
0,"Muere Toni Batllori, autor de la tira diaria d...",Toni Batllori Obiols ha fallecido esta noche e...,Culture,144,1100,11
1,La anarquía explicada a los niños por José A. ...,"En 1931 el maestro José Antonio Emmanuel, y me...",Culture,196,1800,109
2,"Rhianan Rudd, la adolescente a la que el MI5 a...","Rhianan Rudd, una adolescente de 14 años que v...",Latest,330,6100,77
3,Piden por burofax al obispo de Cartagena que r...,"La catedral de Murcia, la iglesia de El Salvad...",Politics,410,881,54
4,Érase una vez... (1950),Érase una vez... (1950) dirigida por Josep Esc...,Culture,114,100,9


# Sentiment Anlysis

In [4]:
!pip install transformers

In [94]:
df_sentiment_try = df.iloc[:]

In [95]:
text_polarity = []
text_score = []

### Create model

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline('sentiment-analysis', 
                      model="nlptown/bert-base-multilingual-uncased-sentiment")

In [96]:
%%time
for text in df_sentiment_try['clean_text']:    
    if len(text) > 512:
        text = text[:512] # max of 512, otherwise, gives error.
    else:
        pass
    results = classifier(text)
    
    # get the polarities
    polarity = results[0]['label']
    text_polarity.append(polarity)
    

    # get the score
    score = round(results[0]['score'], 2)
    text_score.append(score)

CPU times: user 21h 10min 25s, sys: 9min 18s, total: 21h 19min 44s
Wall time: 5h 21min 19s


### Add results to the dataframe

In [98]:
df_sentiment_try['polarity'] = text_polarity
df_sentiment_try['polarity_score'] = text_score

In [97]:
len(text_polarity), len(text_score)

(123500, 123500)

# Save file

In [103]:
df_sentiment_try.to_csv('../DATA/news_125k-tokens-relevant-with_polarity.csv')

### Using TextBlob

In [ ]:
!pip install -U textblob

In [ ]:
!python -m textblob.download_corpora

In [20]:
from textblob import TextBlob

In [ ]:
textblob_polarity = []
textblob_subjectivity = []


for text in df['clean_text']:
    # first we need to translate 
    text_blob = TextBlob(text)
    text_blob.translate(to='en')
    
    # get the polarities
    text_polarity = text_blob.sentiment.polarity
    textblob_polarity.append(text_polarity)
    
    # get the subjectivities
    text_subjectivity = text_blob.sentiment.subjectivity
    textblob_subjectivity.append(text_subjectivity)

In [ ]:
df['polarity'] = textblob_polarity
df['subjectivity'] = textblob_subjectivity